In [1]:
import os, sys
import argparse
import time

import xml.etree.ElementTree as ET

In [2]:
if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

import traci
import traci.constants as tc

net_xml = "seattle.net.xml"
add_xml = "seattle.add.xml"
rou_xml = "seattle.trips.xml"

In [3]:
import curbside
import utils

In [4]:
def main():
    """
    Main function that controls the simulation
    """

    road_network = utils.create_graph('seattle.net.xml')
    
    # create agent curbs
    curb_ids = []
    root = ET.parse(add_xml).getroot()
    for child in root.iter('additional'):
        for kid in child.iter('parkingArea'):
            curb_ids.append(kid.get('id'))
    
    curbs = []
    for curb_id in curb_ids:
        curbs.append(curbside.SmartCurbside(1, 'seattle.add.xml', 'seattle.net.xml', curb_id, ['passenger', 'taxi'], road_network))
        
    for curb in curbs:
        curb.find_neighborhood(road_network)
    
    start = time.time()
    # simulation
    while traci.simulation.getMinExpectedNumber() > 0:
        traci.simulationStep()
        if traci.simulation.getTime() == 1200:
            print(time.time() - start)
            break
        
        for curb in curbs:
            # update neighbors info
            curb._update_nearby_curb()
            
            # check if there is any vehicle enters
            v_enter = set(traci.edge.getLastStepVehicleIDs(curb.edge)) - curb.moving_vehicle
            
            # no need to subtract v_leave, because v_leave ideally should be maintained per curb, but now inner loop is on curbs
            # v_enter -= v_leave
            
            for veh in v_enter:
                # new vehicle is for sure not confirmed, no need to check confirmed=True
                # if vehicle has next stops and currect edge is one of them - this vehicle should be considered for current edge
                # also check vehicle is not stopped - during parking right before departure getNextStops() will return original travel plan, which shouldn't happen
                if traci.vehicle.getNextStops(veh) and curb.id in [item[2] for item in traci.vehicle.getNextStops(veh)] and not traci.vehicle.isStopped(veh):
                    # if the trips is not ending and the current edge is the parking stop
                    # v_leave should be excluded
                    if any(traci.vehicle.getVehicleClass(veh) in s for s in curb.vclass) and len(curb.occupied_vehicle) < curb.capacity:
                        # if can park, add to occupied set : planned + parked
                        curb.occupied_vehicle.add(veh)
                    else:
                        # cannot park at this edge, reroute
                        # item._reroute_choice() returns (curb_id, distance) tuple
                        traci.vehicle.rerouteParkingArea(veh, curb._reroute_choice()[0])
            
            # v_leave : parked vehicle of last time step - parked vehicle at this time step
            v_leave = curb.parked_vehicle - set(traci.parkingarea.getVehicleIDs(curb.id))
            # remove v_leave from occupied set : planned + parked
            curb.occupied_vehicle -= v_leave
            # update parked vehicle set
            curb.parked_vehicle = set(traci.parkingarea.getVehicleIDs(curb.id))
            # update moving vehicles on the hosting edge
            curb.moving_vehicle = set(traci.edge.getLastStepVehicleIDs(curb.edge))

    # enter False to disconnect before SUMO finishes
    traci.close()

    # flush out intermmediate results
    sys.stdout.flush()
    
    
    # backup
    ## junction listen
    ### traci.junction.subscribeContext('01-24', tc.CMD_GET_VEHICLE_VARIABLE, 100, 
                                ## [tc.VAR_TYPE, tc.VAR_ROUTE_ID, tc.VAR_LENGTH, 
                                ## tc.VAR_POSITION, tc.VAR_ROAD_ID, tc.VAR_LANEPOSITION])
    
    # retrive junction listen
    ### if traci.junction.getContextSubscriptionResults('gneJ5'):
        ### print(traci.junction.getContextSubscriptionResults('gneJ5'))

In [5]:
traci.start(["sumo-gui", "-c", "seattle.sumocfg", "--time-to-teleport", "-1"])
main()

 Retrying in 1 seconds
161.5682282447815


In [ ]:
traci.close()

# flush out intermmediate results
sys.stdout.flush()